Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os
import pandas as pd

Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [ ]:
dataset = fo.Dataset.from_images_dir('C:\\AIC Data\\All_Keyframes\\keyframes\\', name=None, tags=None, recursive=True)
dataset.save()

Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [ ]:
# dataset = fo.load_dataset('2024.08.31.11.07.00')
# session = fo.launch_app(dataset, auto=False)
# session.open_tab()

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [ ]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()

In [ ]:
map_keyframes_path = "C:\\AIC Data\\map-keyframes-b1\\map-keyframes\\"
processed_sample_count = 0

for i in range(1, 13):
    csv_files = glob(os.path.join(map_keyframes_path, f"L{i:02}*.csv"))
    ptsTime_frameIdx = {}

    for file in csv_files:
        dataframe = pd.read_csv(file)
        for _, row in dataframe.iterrows():
            key = os.path.splitext(os.path.basename(file))[0] + f"_{int(row['n']):03d}"
            ptsTime_frameIdx[key] = f"{row['pts_time']}-{int(row['frame_idx']):d}"

    processing_count = len(ptsTime_frameIdx)
    for sample in dataset[processed_sample_count: processed_sample_count + processing_count]:
        key = f"{sample['video']}_{sample['frameid']}"
        sample['pts_time, frame_index'] = ptsTime_frameIdx.get(key, "")
        sample.save()

    processed_sample_count += processing_count

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [ ]:
print(dataset.first())

### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
# for sample in dataset:
#     object_path = f"D:\\AICBaseline\\objects-sample\\objects\\{sample['video']}\\{sample['frameid']}.json"
#     with open(object_path) as jsonfile:
#         det_data = json.load(jsonfile)
#     detections = []
#     for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
#         # Convert to [top-left-x, top-left-y, width, height]
#         boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
#         scoref = float(score)

#         # Only add objects with confidence > 0.4
#         if scoref > 0.4:
#             detections.append(
#                 fo.Detection(
#                     label=cls,
#                     bounding_box= boxf,
#                     confidence=float(score)
#                 )
#             )
#     sample["object_faster_rcnn"] = fo.Detections(detections=detections)
#     sample.save()


### Thêm thông tin CLIP embedding.

In [ ]:
video_keyframe_dict = {}
all_keyframe = glob('C:\\AIC Data\\All_Keyframes\\keyframes\\*\\*.jpg')
all_video = glob('C:\\AIC Data\\All_Keyframes\\keyframes\\*')
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

In [ ]:
print(video_keyframe_dict)

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = f'C:\\AIC Data\\clip-features-32-b1\\clip-features-32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [ ]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [ ]:
embedding_dict.keys()

In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
fob.compute_visualization(
    dataset,
    embeddings=clip_embeddings,
    brain_key="img_viz"
)


In [ ]:
datasets = fo.list_datasets()
print(datasets)

In [ ]:
dataset.export(
    export_dir="D:/AO THAT DAY!",
    dataset_type=fo.types.FiftyOneDataset,
)